In [1]:
import numpy as np
import pandas as pd
import requests, time
from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')

In [2]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver', options=options)

In [3]:
songIds = []
base_url = 'https://www.melon.com/genre/song_list.htm?params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&params%5BdtlGnrCode%5D=params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&gnrCode=GN0'
genre_url = '#params%5BgnrCode%5D=GN0'
index_url = '&startIndex='
for i in tqdm(range(100, 801, 100)):
    main_url = f'{base_url}{i}{genre_url}{i}&params%5BdtlGnrCode%5D=&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj'
    for k in range(1, 452, 50):
        page_url = f'{main_url}{index_url}{k}'
        driver.get(page_url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        trs = soup.select('tbody > tr')
        for tr in trs:
            try:
                if tr.select_one('.bullet_icons.age_19'):
                    continue
                songId = tr.select_one('.btn.button_icons.type03.song_info')['href'].split("'")[-2]
                songIds.append(songId)
            except:
                print(page_url)

100%|██████████| 8/8 [03:22<00:00, 25.34s/it]


In [5]:
len(songIds)

3970

songIds[1800:1805]
['30672385', '30715020', '33425479', '36134783', '1755551']

In [20]:
infos = []
for songId in tqdm(songIds[1800:2100]):
    try:
        song_url = f'https://www.melon.com/song/detail.htm?songId={songId}'
        driver.get(song_url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        info = soup.select_one('#conts')
        songId = info.select_one('#btnLike')['data-song-no']
        title = info.select_one('.song_name').get_text().strip().split('\t')[-1]
        artist = info.select_one('.artist_name').get_text()
        genre = info.select_one('dl > dd:nth-child(6)').get_text().replace(',', '')
        album = info.select_one('.list > dd').get_text()
        ul = info.select('.list_person.clfix > li')
        lyricist = ul[0].select_one('.artist_name').get_text()
        composer = ul[1].select_one('.artist_name').get_text()
        date = info.select_one('dl > dd:nth-child(4)').get_text().replace('.','')
        img = info.select_one('.image_typeAll > img')['src']
        comment = info.select_one('#revCnt').get_text().replace('개', '')
        like = info.select_one('#d_like_count').get_text().replace(',', '')
        lyric = str(info.select_one('.lyric')).split('\t\t\t')[-1].replace('\t', '').replace('<br/>', '\n').split('</div>')[0].strip()
        try:
            buttons = soup.select('.meta > button')
            plylstSeq = ' '.join([button['data-djcol-no'] for button in buttons])
            infos.append({'songId': songId, 'title': title, 'artist': artist, 'genre':genre,
                        'album': album, 'lyricist': lyricist, 'composer': composer, 'date':date, 'img': img,
                        'comment': comment, 'like': like, 'lyric': lyric, 'plylstSeq': plylstSeq})
        except:
            plylstSeq = np.nan
            infos.append({'songId': songId, 'title': title, 'artist': artist, 'genre':genre,
                        'album': album, 'lyricist': lyricist, 'composer': composer, 'date':date, 'img': img,
                        'comment': comment, 'like': like, 'lyric': lyric, 'plylstSeq': plylstSeq})

    except:
        continue

100%|██████████| 300/300 [17:04<00:00,  3.42s/it]


In [21]:
df = pd.DataFrame(infos)
df.shape[0]

297

In [22]:
df.to_csv('data/melon_genre7.csv', index=False)
pd.read_csv('data/melon_genre7.csv').head(2)

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq
0,36081382,친구로 지내다 보면 (Feat. 김민석 of 멜로망스),BIG Naughty (서동현),R&B/Soul,친구로 지내다 보면,BIG Naughty (서동현),dress,20230129,https://cdnimg.melon.co.kr/cm2/album/images/11...,267,41335,친구로 지내다 보면\n계속 네 옆에 있다 보면\n언젠가는 나에게도\n기회가 오지 않...,457417618 458920936
1,32243273,Square (2017),백예린 (Yerin Baek),R&B/Soul,Every letter I sent you.,백예린 (Yerin Baek),백예린 (Yerin Baek),20191210,https://cdnimg.melon.co.kr/cm2/album/images/10...,"2,774",276495,NaN,478102836 411111859
